# Loop 4 Analysis: Understanding Group 008 Overlap Issue

All 3 submissions failed with 'Overlapping trees in group 008'. The LB uses:
- `intersects() and not touches()` for overlap detection
- `scale_factor = 1e15` for coordinates (Decimal precision)

Let me analyze exactly what's happening.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Match LB precision exactly
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded')

In [ ]:
# Tree geometry - EXACTLY as in the LB metric
trunk_w = Decimal('0.15')
trunk_h = Decimal('0.2')
base_w = Decimal('0.7')
mid_w = Decimal('0.4')
top_w = Decimal('0.25')
tip_y = Decimal('0.8')
tier_1_y = Decimal('0.5')
tier_2_y = Decimal('0.25')
base_y = Decimal('0.0')
trunk_bottom_y = -trunk_h

def create_tree_polygon_lb(center_x_str, center_y_str, angle_str):
    """Create tree polygon EXACTLY as the LB does - using Decimal and scale_factor."""
    center_x = Decimal(center_x_str)
    center_y = Decimal(center_y_str)
    angle = Decimal(angle_str)
    
    initial_polygon = Polygon([
        (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
        (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
        (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
        (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
        (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
        (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
        (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
        (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
        (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
        (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
        (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
        (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
        (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
        (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
        (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
    ])
    
    rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
    polygon = affinity.translate(rotated,
                                 xoff=float(center_x * scale_factor),
                                 yoff=float(center_y * scale_factor))
    return polygon

print('Tree polygon function defined (LB-compatible)')

In [ ]:
def check_lb_overlaps(polygons):
    """Check for overlaps using EXACT LB method."""
    overlapping_pairs = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlapping_pairs.append((i, j))
    return overlapping_pairs

print('LB overlap check defined')

In [ ]:
# Load and parse the submission that failed
print('Loading failed submission...')
failed_df = pd.read_csv('/home/submission/submission.csv')

# Parse group 008
group_008 = failed_df[failed_df['id'].str.startswith('008_')].copy()
print(f'Group 008 has {len(group_008)} trees')
print(group_008)

In [ ]:
# Create polygons for group 008 using LB method
print('\nCreating polygons using LB method...')
polygons_008 = []
for _, row in group_008.iterrows():
    x = row['x'].replace('s', '')
    y = row['y'].replace('s', '')
    deg = row['deg'].replace('s', '')
    poly = create_tree_polygon_lb(x, y, deg)
    polygons_008.append(poly)
    print(f"{row['id']}: x={x[:20]}..., y={y[:20]}..., deg={deg[:20]}...")

print(f'\nCreated {len(polygons_008)} polygons')

In [ ]:
# Check for overlaps
print('\nChecking for LB overlaps...')
overlapping = check_lb_overlaps(polygons_008)
if overlapping:
    print(f'FOUND {len(overlapping)} overlapping pairs!')
    for i, j in overlapping:
        print(f'  Trees {i} and {j} overlap')
        # Check details
        p1, p2 = polygons_008[i], polygons_008[j]
        print(f'    intersects: {p1.intersects(p2)}')
        print(f'    touches: {p1.touches(p2)}')
        print(f'    intersection area: {p1.intersection(p2).area}')
        print(f'    distance: {p1.distance(p2)}')
else:
    print('No overlaps found!')

In [ ]:
# Now check chistyakov_best.csv group 008
print('\n' + '='*60)
print('Checking chistyakov_best.csv group 008...')
print('='*60)

chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_008 = chistyakov_df[chistyakov_df['id'].str.startswith('008_')].copy()
print(f'Group 008 has {len(chistyakov_008)} trees')
print(chistyakov_008)

In [ ]:
# Create polygons for chistyakov group 008
print('\nCreating polygons for chistyakov group 008...')
chistyakov_polys = []
for _, row in chistyakov_008.iterrows():
    x = row['x'].replace('s', '')
    y = row['y'].replace('s', '')
    deg = row['deg'].replace('s', '')
    poly = create_tree_polygon_lb(x, y, deg)
    chistyakov_polys.append(poly)

# Check for overlaps
overlapping = check_lb_overlaps(chistyakov_polys)
if overlapping:
    print(f'FOUND {len(overlapping)} overlapping pairs!')
    for i, j in overlapping:
        print(f'  Trees {i} and {j} overlap')
else:
    print('No overlaps found in chistyakov group 008!')

In [ ]:
# Check ALL groups in chistyakov_best.csv
print('\n' + '='*60)
print('Checking ALL groups in chistyakov_best.csv...')
print('='*60)

overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_df[chistyakov_df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        x = row['x'].replace('s', '')
        y = row['y'].replace('s', '')
        deg = row['deg'].replace('s', '')
        poly = create_tree_polygon_lb(x, y, deg)
        polygons.append(poly)
    
    overlapping = check_lb_overlaps(polygons)
    if overlapping:
        overlap_groups.append(n)
        if n <= 20:  # Only print first few
            print(f'N={n}: {len(overlapping)} overlapping pairs')

print(f'\nTotal groups with overlaps: {len(overlap_groups)}')
if overlap_groups:
    print(f'Overlap groups: {overlap_groups[:20]}...')

In [ ]:
# Check submission_70_926.csv (should be same as chistyakov)
print('\n' + '='*60)
print('Checking submission_70_926.csv...')
print('='*60)

sub_926_df = pd.read_csv('/home/code/preoptimized/submission_70_926.csv')

overlap_groups_926 = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = sub_926_df[sub_926_df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in config_df.iterrows():
        x = row['x'].replace('s', '')
        y = row['y'].replace('s', '')
        deg = row['deg'].replace('s', '')
        poly = create_tree_polygon_lb(x, y, deg)
        polygons.append(poly)
    
    overlapping = check_lb_overlaps(polygons)
    if overlapping:
        overlap_groups_926.append(n)

print(f'Total groups with overlaps: {len(overlap_groups_926)}')
if overlap_groups_926:
    print(f'Overlap groups: {overlap_groups_926[:20]}...')